In [1]:
import json
import pandas as pd
import datetime as dt

with open('data/1066392.json', encoding='utf-8') as f:
    d = json.load(f)
    
df = pd.json_normalize(d['members'].values())

df = df.drop(['last_star_ts', 'global_score', 'id'], axis=1)
left, right = 'completion_day_level.', '.get_star_ts'
df = df.rename(columns={col: col.lstrip(left).rstrip(right) for col in df.columns if col.startswith(left)})
df = df.rename(columns={'local_score': 'score'})
df = df[['name', 'score', 'stars'] + list(sorted(df.columns[3:]))]

conv_temps = lambda x: pd.NaT if pd.isna(x) or x==0 else dt.datetime.fromtimestamp(int(x))

for column in df.columns[3:]:
    df[column] = df[column].apply(conv_temps)

df['acumulat'] = dt.timedelta()
for col in range(3, df.shape[1]-1, 2):
    df[df.columns[col+1]] -= df[df.columns[col]]
    df['acumulat'] += df[df.columns[col+1]]
    df[df.columns[col]] = df[df.columns[col]].dt.strftime('%d-%m %H:%M:%S')

df = df.sort_values(by='acumulat', ascending=True)

pd.set_option('max_columns', None)
styles = [dict(selector='th', props=[('font-size', '150%'), ('text-align', 'center')]),
          dict(selector='td', props=[('text-align', 'center')])]

df.style.set_table_styles(styles).hide_index()

name,score,stars,1.1,1.2,2.1,2.2,3.1,3.2,4.1,4.2,5.1,5.2,6.1,6.2,acumulat
Felipe Mourad Pereira,214,12,02-12 06:39:18,0 days 00:02:50,02-12 06:59:00,0 days 00:02:28,03-12 06:09:48,0 days 00:05:04,04-12 06:29:16,0 days 00:25:39,05-12 06:08:29,0 days 00:02:16,06-12 06:02:37,0 days 00:03:52,0 days 00:42:09
David Rial Figols,167,12,02-12 15:09:22,0 days 00:00:48,02-12 12:11:09,0 days 00:03:15,03-12 08:29:21,0 days 00:07:43,04-12 09:13:30,0 days 00:22:13,05-12 11:18:53,0 days 00:07:56,06-12 10:53:20,0 days 00:07:34,0 days 00:49:29
Andreu Vall Hernàndez,131,12,02-12 13:45:24,0 days 00:01:34,02-12 13:28:12,0 days 00:09:45,03-12 10:28:45,0 days 00:04:43,04-12 11:12:00,0 days 00:33:48,05-12 12:30:40,0 days 00:06:07,06-12 12:01:26,0 days 00:03:40,0 days 00:59:37
albertsalles4,134,12,02-12 12:44:23,0 days 00:01:45,02-12 13:12:22,0 days 00:07:45,03-12 11:30:32,0 days 00:03:12,04-12 11:09:57,0 days 00:52:10,05-12 11:33:49,0 days 00:06:33,06-12 11:38:15,0 days 00:02:06,0 days 01:13:31
Carla Morral Martin,208,12,01-12 21:48:48,0 days 00:11:03,02-12 06:07:31,0 days 00:05:09,03-12 06:21:34,0 days 00:03:36,04-12 07:12:13,0 days 00:30:16,05-12 06:54:57,0 days 00:17:33,06-12 06:07:41,0 days 00:13:45,0 days 01:21:22
Georg Reich,206,12,01-12 06:07:36,0 days 00:04:05,02-12 06:03:17,0 days 00:04:17,03-12 06:05:13,0 days 00:08:36,04-12 07:23:12,0 days 00:37:15,05-12 08:01:36,0 days 00:23:16,06-12 08:42:25,0 days 00:05:20,0 days 01:22:49
sarabase,153,12,02-12 13:17:23,0 days 00:01:45,02-12 13:38:29,0 days 00:05:26,03-12 10:41:43,0 days 00:05:03,04-12 09:40:55,0 days 01:13:47,05-12 10:20:16,0 days 00:06:52,06-12 09:35:36,0 days 00:18:00,0 days 01:50:53
Arnau Q,163,12,02-12 00:22:10,0 days 00:01:03,02-12 10:04:31,0 days 00:08:09,03-12 10:25:56,0 days 00:06:18,04-12 17:38:16,0 days 01:12:02,05-12 10:06:21,0 days 00:07:00,06-12 11:00:32,0 days 00:24:42,0 days 01:59:14
laurentiuNedelcu,163,12,02-12 01:38:57,0 days 00:27:22,02-12 10:28:11,0 days 00:17:53,03-12 10:55:05,0 days 00:09:20,04-12 09:50:21,0 days 01:19:30,05-12 10:59:40,0 days 00:37:21,06-12 09:21:28,0 days 00:36:59,0 days 03:28:25
Lluís Camino,60,8,01-12 23:38:43,0 days 00:19:19,02-12 13:34:07,0 days 00:07:05,03-12 21:50:23,0 days 00:09:10,nan,NaT,06-12 00:48:17,0 days 00:11:26,nan,NaT,NaT
